# Сборный проект-4

Нам поручено разработать демонстрационную версию поиска изображений по запросу.

Для демонстрационной версии нужно обучить модель, которая получит векторное представление изображения, векторное представление текста, а на выходе выдаст число от 0 до 1 — покажет, насколько текст и картинка подходят друг другу.



## Описание данных





Данные доступны по [ссылке](https://code.s3.yandex.net/datasets/dsplus_integrated_project_4.zip).

В файле `train_dataset.csv` находится информация, необходимая для обучения: имя файла изображения, идентификатор описания и текст описания. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `train_images` содержатся изображения для тренировки модели.

В файле `CrowdAnnotations.tsv` — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.
3. Доля людей, подтвердивших, что описание соответствует изображению.
4. Количество человек, подтвердивших, что описание соответствует изображению.
5. Количество человек, подтвердивших, что описание не соответствует изображению.

В файле `ExpertAnnotations.tsv` содержатся данные по соответствию изображения и описания, полученные в результате опроса экспертов. Номера колонок и соответствующий тип данных:

1. Имя файла изображения.
2. Идентификатор описания.

3, 4, 5 — оценки трёх экспертов.

Эксперты ставят оценки по шкале от 1 до 4, где 1 — изображение и запрос совершенно не соответствуют друг другу, 2 — запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует, 3 — запрос и текст соответствуют с точностью до некоторых деталей, 4 — запрос и текст соответствуют полностью.

В файле `test_queries.csv` находится информация, необходимая для тестирования: идентификатор запроса, текст запроса и релевантное изображение. Для одной картинки может быть доступно до 5 описаний. Идентификатор описания имеет формат `<имя файла изображения>#<порядковый номер описания>`.

В папке `test_images` содержатся изображения для тестирования модели.

Импортируем необходимые для работы библиотеки

In [1]:
!pip install torchvision


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

Клонируем репозиторий с гитхаба

In [3]:
!git clone https://github.com/gala-vision/project_15.git
%cd project_15/notebooks


C:\Users\Галина\project_15\notebooks


fatal: destination path 'project_15' already exists and is not an empty directory.


In [4]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
src_path = os.path.join(os.getcwd(), os.pardir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
print("Путь к src:", src_path)


Путь к src: C:\Users\Галина\project_15\notebooks\..\src


In [5]:
from src.utils import information_data, aggregate_expert_ratings

## 1. Исследовательский анализ данных

Датасет содержит экспертные и краудсорсинговые оценки соответствия текста и изображения.

В файле с экспертными мнениями для каждой пары изображение-текст имеются оценки от трёх специалистов. Для решения задачи нужно эти оценки агрегировать — превратить в одну. Существует несколько способов агрегации оценок, самый простой — голосование большинства: за какую оценку проголосовала большая часть экспертов (в нашем случае 2 или 3), та оценка и ставится как итоговая. Поскольку число экспертов меньше числа классов, может случиться, что каждый эксперт поставит разные оценки, например: 1, 4, 2. В таком случае данную пару изображение-текст исключим из датасета.

В файле с краудсорсинговыми оценками информация расположена в таком порядке:

1. Доля исполнителей, подтвердивших, что текст **соответствует** картинке.
2. Количество исполнителей, подтвердивших, что текст **соответствует** картинке.
3. Количество исполнителей, подтвердивших, что текст **не соответствует** картинке.

После анализа экспертных и краудсорсинговых оценок объединим их в одну по критерию: оценка эксперта принимается с коэффициентом 0.6, а крауда — с коэффициентом 0.4 и нормализуем. Если оценки экспертов нет, будем использовать оценку крауда из `fraction_confirmed`, она уже нормализована (от 0 до 1).

Модель будет возвращать на выходе вероятность соответствия изображения тексту, поэтому целевая переменная имеет значения от 0 до 1.


### `train_queries`

In [6]:
data_path = os.path.join(os.getcwd(), os.pardir, 'data')
train_images_path = os.path.join(data_path, 'train_images')
test_images_path = os.path.join(data_path, 'test_images')

In [7]:
train_queries = pd.read_csv(os.path.join(data_path, 'train_dataset.csv'))

print('Тренировочные запросы', end='\n\n')
information_data(train_queries)

Тренировочные запросы

Первые десять строк датафрейма:


,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
5,3030566410_393c36a6c5.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
6,3155451946_c0862c70cb.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
7,3222041930_f642f49d28.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
8,343218198_1ca90e0734.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
9,3718964174_cb2dc1615e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...


Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB


None

Описание данных:


,image,query_id,query_text
count,5822,5822,5822
unique,1000,977,977
top,3364151356_eecd07a23e.jpg,2600867924_cd502fc911.jpg#2,"Two dogs , one brown and white and one black a..."
freq,10,34,34


Количество пропусков:


image         0
query_id      0
query_text    0
dtype: int64

Количество дубликатов:


0

### `test_queries`

In [8]:
test_queries = pd.read_csv(os.path.join(data_path, 'test_queries.csv'), sep='|', index_col=0)

print('Тестовые запросы', end='\n\n')
information_data(test_queries)

Тестовые запросы

Первые десять строк датафрейма:


,query_id,query_text,image
0,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
1,1177994172_10d143cb8d.jpg#1,Two boys are squirting water guns at each other .,1177994172_10d143cb8d.jpg
2,1177994172_10d143cb8d.jpg#2,Two boys spraying each other with water,1177994172_10d143cb8d.jpg
3,1177994172_10d143cb8d.jpg#3,Two children wearing jeans squirt water at eac...,1177994172_10d143cb8d.jpg
4,1177994172_10d143cb8d.jpg#4,Two young boys are squirting water at each oth...,1177994172_10d143cb8d.jpg
5,1232148178_4f45cc3284.jpg#0,A baby girl playing at a park .,1232148178_4f45cc3284.jpg
6,1232148178_4f45cc3284.jpg#1,A closeup of a child on a playground with adul...,1232148178_4f45cc3284.jpg
7,1232148178_4f45cc3284.jpg#2,A young boy poses for a picture in front of a ...,1232148178_4f45cc3284.jpg
8,1232148178_4f45cc3284.jpg#3,A young girl is smiling in front of the camera...,1232148178_4f45cc3284.jpg
9,1232148178_4f45cc3284.jpg#4,There is a little blond hair girl with a green...,1232148178_4f45cc3284.jpg


Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   query_id    500 non-null    object
 1   query_text  500 non-null    object
 2   image       500 non-null    object
dtypes: object(3)
memory usage: 15.6+ KB


None

Описание данных:


,query_id,query_text,image
count,500,500,500
unique,500,500,100
top,1177994172_10d143cb8d.jpg#0,"Two blonde boys , one in a camouflage shirt an...",1177994172_10d143cb8d.jpg
freq,1,1,5


Количество пропусков:


query_id      0
query_text    0
image         0
dtype: int64

Количество дубликатов:


0

Обе выборки (обучающая и тестовая) имеют одинаковую структуру – содержат три столбца: идентификатор запроса (query_id), текст запроса (query_text) и название изображения (image).

Тестовая выборка:

Состоит из 500 записей, все значения уникальны.
Пропусков и дубликатов нет, что обеспечивает чистоту данных для тестирования модели.

Обучающая выборка:

Включает 5822 записи.
Количество уникальных значений немного отличается: имеется 1000 уникальных изображений и 977 уникальных запросов.
Также не содержит пропусков или дубликатов, что гарантирует корректное обучение модели.
Таким образом, обе выборки структурно идентичны и готовы для дальнейшей обработки при обучении и тестировании модели.

### `crowd_df`

In [9]:
column_names = ['image', 'query_id', 'fraction_confirmed', 'num_confirmed', 'num_rejected']
crowd_df = pd.read_csv(os.path.join(data_path, 'CrowdAnnotations.tsv'), sep='\t', names=column_names,
                       header=0)

print('Краудсорсинговые оценки', end='\n\n')
information_data(crowd_df)


Краудсорсинговые оценки

Первые десять строк датафрейма:


,image,query_id,fraction_confirmed,num_confirmed,num_rejected
0,1056338697_4f7d7ce270.jpg,114051287_dd85625a04.jpg#2,0.000000,0,3
1,1056338697_4f7d7ce270.jpg,1427391496_ea512cbe7f.jpg#2,0.000000,0,3
2,1056338697_4f7d7ce270.jpg,2073964624_52da3a0fc4.jpg#2,0.000000,0,3
3,1056338697_4f7d7ce270.jpg,2083434441_a93bc6306b.jpg#2,0.000000,0,3
4,1056338697_4f7d7ce270.jpg,2204550058_2707d92338.jpg#2,0.000000,0,3
5,1056338697_4f7d7ce270.jpg,2224450291_4c133fabe8.jpg#2,0.000000,0,3
6,1056338697_4f7d7ce270.jpg,2248487950_c62d0c81a9.jpg#2,0.333333,1,2
7,1056338697_4f7d7ce270.jpg,2307118114_c258e3a47e.jpg#2,0.000000,0,3
8,1056338697_4f7d7ce270.jpg,2309860995_c2e2a0feeb.jpg#2,0.000000,0,3
9,1056338697_4f7d7ce270.jpg,2436081047_bca044c1d3.jpg#2,0.000000,0,3


Общая информация о датафрейме:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47829 entries, 0 to 47828
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   image               47829 non-null  object 
 1   query_id            47829 non-null  object 
 2   fraction_confirmed  47829 non-null  float64
 3   num_confirmed       47829 non-null  int64  
 4   num_rejected        47829 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.8+ MB


None

Описание данных:


,fraction_confirmed,num_confirmed,num_rejected
count,47829.000000,47829.000000,47829.000000
mean,0.068766,0.208430,2.820214
std,0.207490,0.628775,0.656556
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,3.000000
50%,0.000000,0.000000,3.000000
75%,0.000000,0.000000,3.000000
max,1.000000,5.000000,6.000000


Количество пропусков:


image                 0
query_id              0
fraction_confirmed    0
num_confirmed         0
num_rejected          0
dtype: int64

Количество дубликатов:


0

In [ ]:
# Настройки графиков
sns.set(style="whitegrid")

# Графики для crowd_df
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(crowd_df['fraction_confirmed'], bins=20, kde=True, ax=axes[0])
axes[0].set_title('Распределение fraction_confirmed')

sns.histplot(crowd_df['num_confirmed'], bins=range(0, crowd_df['num_confirmed'].max() + 1), kde=False, ax=axes[1])
axes[1].set_title('Распределение num_confirmed')

sns.histplot(crowd_df['num_rejected'], bins=range(0, crowd_df['num_rejected'].max() + 1), kde=False, ax=axes[2])
axes[2].set_title('Распределение num_rejected')

plt.tight_layout()
plt.show()

### `expert_df`

In [ ]:
column_names = ['image', 'query_id', 'expert_1', 'expert_2', 'expert_3']
expert_df = pd.read_csv(os.path.join(data_path, 'ExpertAnnotations.tsv'), sep='\t', names=column_names, header=0)

print('Оценки экспертов', end='\n\n')
information_data(expert_df)

In [ ]:
# Графики для expert_df
fig, ax = plt.subplots(figsize=(8, 5))

expert_scores = expert_df[['expert_1', 'expert_2', 'expert_3']].values.flatten()
sns.histplot(expert_scores, bins=range(1, expert_scores.max() + 2), kde=False)
ax.set_title('Распределение оценок экспертов')
ax.set_xticks(range(1, expert_scores.max() + 1))


Проверим связь между `query_id` и `image`:

Для этого разделим `query_id` по `#` и выделяем `image`.
Проверим, есть ли несовпадения между `image` в `query_id` и выделенным `image_from_query`.

И проверим полноту соответствий между изображениями и запросами: есть ли `image`, которые не встречаются в `query_id`, и наоборот.

In [ ]:
crowd_df['image_from_query'] = crowd_df['query_id'].str.split('#').str[0]
expert_df['image_from_query'] = expert_df['query_id'].str.split('#').str[0]

# Проверяем, совпадают ли image и query_image
crowd_mismatch = crowd_df[crowd_df['image_from_query'] != crowd_df['image_from_query']]
expert_mismatch = expert_df[expert_df['image_from_query'] != expert_df['image_from_query']]

print(f'Несовпадения в CrowdAnnotations: {len(crowd_mismatch)}')
print(f'Несовпадения в ExpertAnnotations: {len(expert_mismatch)}')

# Проверяем, есть ли images, которых нет в query_id, и наоборот
unique_images_crowd = set(crowd_df['image_from_query'].unique())
unique_queries_crowd = set(crowd_df['image_from_query'].unique())
missing_images_crowd = unique_queries_crowd - unique_images_crowd

unique_images_expert = set(expert_df['image_from_query'].unique())
unique_queries_expert = set(expert_df['image_from_query'].unique())
missing_images_expert = unique_queries_expert - unique_images_expert

print(f'Изображений без query_id в CrowdAnnotations: {len(missing_images_crowd)}')
print(f'Изображений без query_id в ExpertAnnotations: {len(missing_images_expert)}')

Заключение по датасетам оценок:

В датасете краудсорсинговых оценок представлено 47 829 записей, каждая из которых содержит имя изображения, идентификатор описания и статистику подтверждения соответствия (доля подтверждений, число подтверждений и число отказов). Значения подтверждения в большинстве случаев равны нулю, что указывает на низкую согласованность оценок для большинства пар «изображение–описание».

В датасете экспертных оценок содержится 5 821 запись с тремя оценками от экспертов по шкале от 1 до 4. Экспертные оценки, в отличие от краудсорсинговых, характеризуются более высокой согласованностью и, в среднем, ниже значениями (близкими к минимальному порогу), что свидетельствует о строгом подходе экспертов к оценке соответствия.

Оба датасета не содержат пропусков и дубликатов, что гарантирует корректность данных для последующего анализа и обучения модели.

Несовпадений нет (все `image` правильно извлекаются из `query_id`).

Все изображения имеют `query_id`, пропусков нет.
Вывод: данные корректны, `image` и `query_id` связаны ожидаемым образом.

### `final_df`

Агрегируем оценки соответствия описаний изображениям следующим образом:

Агрегация оценок:

Для пар с экспертными оценками:

Рассчитаем итоговую оценку по голосованию большинства, нормализуем её.
Рассчитаем итоговую оценку как

$ 0.7 × $ `expert_norm`$ + 0.3 × $ `fraction_confirmed`

Для пар без экспертных оценок используем крауд-оценку

Объединение датасетов:

Соберем итоговую таблицу, где для каждой пары «изображение–описание» будет указана итоговая оценка. При этом, если требуется, можно добавить отдельные колонки для экспертных и крауд оценок как дополнительные признаки, но итоговая агрегированная оценка должна стать целевой переменной.

In [ ]:
# 1. Агрегируем экспертные оценки
expert_df['expert_score'] = expert_df.apply(aggregate_expert_ratings, axis=1)
# expert_df.drop(columns=['expert_1', 'expert_2', 'expert_3'], inplace=True)  # Удаляем исходные колонки
expert_df.dropna(subset=['expert_score'], inplace=True)  # Убираем строки с противоречивыми оценками

In [ ]:
# 2. Нормализуем экспертные оценки (min=1, max=4 -> от 0 до 1)
expert_df['expert_score_norm'] = (expert_df['expert_score'] - 1) / 3

In [ ]:
# 3. Объединяем датасеты
merged_df = pd.merge(expert_df, crowd_df, on=['image', 'query_id'], how='left')

In [ ]:
# 4. Если есть обе оценки - берем взвешенное среднее
mask = (~merged_df['expert_score_norm'].isna()) & (~merged_df['fraction_confirmed'].isna())
merged_df.loc[mask, 'final_score'] = (
    0.7 * merged_df.loc[mask, 'expert_score_norm'] + 0.3 * merged_df.loc[mask, 'fraction_confirmed']
)

In [ ]:
# 5. Если есть только одна из оценок, используем её
merged_df['final_score'] = merged_df['final_score'].combine_first(
    merged_df['expert_score_norm'].combine_first(merged_df['fraction_confirmed'])
)

In [ ]:
# 6. Оставляем только нужные столбцы
final_df = merged_df[['image', 'query_id', 'final_score']]

Посмотрим на распределение таргета на гистограмме

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(final_df['final_score'], bins=30, kde=True, color='royalblue')
plt.xlabel('Final Score')
plt.ylabel('Count')
plt.title('Distribution of Final Score')
plt.grid(True)
plt.show()


In [ ]:
information_data(final_df)

Итоговый датасет содержит 5695 записей, каждая из которых представляет собой пару изображение-описание с оценкой соответствия (final_score), рассчитанной на основе экспертных и краудсорсинговых оценок.

Общие характеристики датасета:
- Всего строк: 5695
- Количество уникальных изображений: 1000
- Количество уникальных описаний: 977
- Количество пропущенных значений: 0
- Количество дубликатов: 0

Распределение таргета (final_score)
- Средняя оценка: 0.18
- Медиана: 0.00 (то есть большинство описаний слабо соответствуют изображениям)
- Стандартное отклонение: 0.27
- Минимальное значение: 0.00
- Максимальное значение: 1.00
- 25-й процентиль: 0.00
- 75-й процентиль: 0.33

Можно видеть, что медиана равна 0, а 75% значений ниже 0.33, что говорит о том, что значительная часть пар "изображение-описание" была оценена низко. Однако есть небольшое количество примеров с высоким соответствием.

Качество соответствия изображений и описаний

Примеры данных показывают, что некоторые описания полностью не соответствуют изображениям (оценка 0), в то время как другие имеют частичное или полное совпадение (0.33 и выше). Оставим пока в таком виде: обучение на отрицательных примерах также важно, потому что модель должна не только определять соответствие изображения и текста, но и понимать, когда описание не соответствует изображению. Это позволит ей лучше различать релевантные и нерелевантные пары. В дальнейшем можно будет поэспериментировать с порогами классов и понаблюдать, как будет меняться метрика.

Следующие шаги

- Очистить датасет от изображений, содержащих несовершеннолетних, используя текстовые описания и (опционально) предобученные модели.
- Провести финальный анализ очищенного датасета перед векторизацией данных и обучением модели.

Посмотрим несколько изображений и описаний к ним

In [ ]:
sample = train_queries.sample(8, random_state=1000)  # Берем 8 случайных примеров

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, row in enumerate(sample.itertuples()):
    image_path = os.path.join(train_images_path, row.image)

    try:
        image = Image.open(image_path)
        axes[idx].imshow(image)
        axes[idx].axis("off")
        axes[idx].set_title(row.query_text, fontsize=10)
    except FileNotFoundError:
        print(f"Файл {row.image} не найден.")
        axes[idx].axis("off")

plt.tight_layout()
plt.show()

## 2. Проверка данных

В некоторых странах, где работает наша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.

В нашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

> This image is unavailable in your country in compliance with local laws
>

Однако у нас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, удалим из обучающей выборки.

Анализ текстовых описаний

Если в описании явно упоминается ребенок (например, "child", "kid", "baby", "boy", "girl" и т. д.), то, скорее всего, на картинке изображен несовершеннолетний.

Используем NLP (анализ текста) для поиска таких слов в query_text.
Если описание содержит слова, связанные с детьми, то удаляем эту запись.

In [ ]:
# добавим в итоговый датасет текстовое описание изображения и провеврим на дубликаты
final_df_merged = final_df.merge(train_queries, on=['query_id', 'image'], how='left')
final_df_merged.duplicated().sum()

In [ ]:
# проверяем результат
print(final_df_merged.info()) 
display(final_df_merged.head())  

In [ ]:
child_keywords = ["child", "kid", "baby", "boy", "girl", "toddler", "infant", "teen", "young", "little", "small"]

mask = final_df_merged['query_text'].str.contains('|'.join(child_keywords), case=False, na=False)

cleaned_df = final_df_merged[~mask]

print(f"Удалено записей: {final_df_merged.shape[0] - cleaned_df.shape[0]}")


In [ ]:
information_data(cleaned_df)

Проверим, как прошло очищение данных:

In [ ]:
# Список ключевых слов, которые могут указывать на присутствие детей
child_keywords = ["child", "baby", "toddler", "kid", "infant", "boy", "girl", 
                  "young boy", "young girl", "little boy", "little girl", 
                  "children", "kids", "school"]

# Приводим query_text к нижнему регистру и проверяем наличие ключевых слов
mask_children = cleaned_df['query_text'].str.lower().str.contains('|'.join(child_keywords), na=False)

# Отбираем записи, где могут быть дети
potential_child_images = cleaned_df[mask_children]

# Смотрим примеры
display(potential_child_images.head()) 

## 3. Векторизация изображений

Перейдём к векторизации изображений.

Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют "выделить" главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

In [ ]:
# 1. Загружаем предобученную модель ResNet-18
model = models.resnet18(pretrained=True)

# 2. Убираем последний слой (полносвязный)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()  # Переключаем в режим оценки

# 3. Преобразование изображений в тензоры
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Изменяем размер изображения
    transforms.ToTensor(),          # Преобразуем в тензор
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

# 4. Функция для векторизации изображений
def extract_features(image_path, model):
    try:
        image = Image.open(image_path).convert("RGB")
        image = transform(image).unsqueeze(0)  # Добавляем размерность batch
        with torch.no_grad():
            features = model(image)  # Получаем векторное представление
        return features.squeeze().numpy()  # Преобразуем в numpy
    except Exception as e:
        print(f"Ошибка с файлом {image_path}: {e}")
        return None

# 5. Векторизуем изображения из cleaned_df
image_vectors = {}
image_folder = "data/train_images"  # Путь к папке с изображениями

for img_name in cleaned_df['image'].unique():
    img_path = os.path.join(image_folder, img_name)
    if os.path.exists(img_path):
        image_vectors[img_name] = extract_features(img_path, model)

# 6. Преобразуем в DataFrame
image_vectors_df = pd.DataFrame.from_dict(image_vectors, orient='index')
image_vectors_df.reset_index(inplace=True)
image_vectors_df.rename(columns={'index': 'image'}, inplace=True)

# 7. Объединяем с финальным датасетом
final_df_with_vectors = cleaned_df.merge(image_vectors_df, on='image', how='left')

# 8. Сохраняем векторизированные изображения
final_df_with_vectors.to_csv("final_df_with_vectors.csv", index=False)

print("Готово! Изображения векторизированы и сохранены. ✅")


In [ ]:
# 1. Проверяем размерность векторов
# print(f"Размерность векторизованных изображений: {image_vectors.shape}")

# 2. Проверяем, есть ли пропуски
missing_vectors = np.isnan(image_vectors).sum()
print(f"Количество пропущенных значений в векторах: {missing_vectors}")

# 3. Выводим несколько примеров векторизованных изображений
print("Примеры первых 5 векторов:")
print(image_vectors[:5])

# 4. Базовая статистика
print("\nСтатистика по векторизованным изображениям:")
print(pd.DataFrame(image_vectors).describe())


## 4. Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

- tf-idf
- word2vec
- \*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.


## 5. Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

## 6. Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.
Для того чтобы учесть изображения при разбиении, можно воспользоваться классом [GroupShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) из библиотеки sklearn.model_selection.

Код ниже разбивает датасет на тренировочную и тестовую выборки в пропорции 7:3 так, что строки с одинаковым значением 'group_column' будут содержаться либо в тестовом, либо в тренировочном датасете.

```
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=42)
train_indices, test_indices = next(gss.split(X=df.drop(columns=['target']), y=df['target'], groups=df['group_column']))
train_df, test_df = df.loc[train_indices], df.loc[test_indices]

```

Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.

## 7. Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки `test_images`, выберите случайные 10 запросов из файла `test_queries.csv` и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.

## 8. Выводы

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Исследовательский анализ данных выполнен
- [ ]  Проверены экспертные оценки и краудсорсинговые оценки
- [ ]  Из датасета исключены те объекты, которые выходят за рамки юридических ограничений
- [ ]  Изображения векторизованы
- [ ]  Текстовые запросы векторизованы
- [ ]  Данные корректно разбиты на тренировочную и тестовую выборки
- [ ]  Предложена метрика качества работы модели
- [ ]  Предложена модель схожести изображений и текстового запроса
- [ ]  Модель обучена
- [ ]  По итогам обучения модели сделаны выводы
- [ ]  Проведено тестирование работы модели
- [ ]  По итогам тестирования визуально сравнили качество поиска